In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from dotenv import load_dotenv
load_dotenv();

In [4]:
clarion.server_login()
bq = BlpQuery(timeout=50000, parser=BlpParser(raise_security_errors=False)).start()
# ba = BHSysApi(password="A&3jg5Aj")

2023-03-10 10:18:31,749 - Authenticating...
2023-03-10 10:18:32,255 - Successfully authenticated


In [5]:
super_clarion(clarion)
runner = Runner(clarion)

In [6]:
from ipyaggrid import Grid

In [8]:
### check metrics
pd.concat(clarion.metrics()).to_clipboard()

### Metrics

In [9]:
trades = get_trades(clarion)

In [10]:
targets = get_targets()

In [11]:
clarion_metrics = dict(
    pv = clarion.metric('pv', 'PresentValue', 'Any'),
    local_pv = clarion.metric('local_pv', 'LocalPresentValue', 'Any'),
    # currency = clarion.metric('currency', 'PayCurrency', 'Any'),
    strike = clarion.metric('strike', 'Strike', 'Any'),
    notional = clarion.metric('notional', 'Notional', 'Any'),
    expiry = clarion.metric('expiry', 'ExpiryDate', 'Any'),
    maturity = clarion.metric('maturity_date', 'MaturityDate', 'Rates', {'Type': 'Bumped'}),
    start = clarion.metric('start_date', 'StartDate', 'Rates', {'Type': 'Bumped'}),
    fwd_rate = clarion.metric('fwd_rate', 'Forward', 'Rates'),
    delta = clarion.metric('delta', 'Delta', 'Rates', {'VolSpotDynamic': 'StickyDelta', 'RiskSpace': 'Par', 'RiskBuckets': 'Default'}),
    gamma = clarion.metric('gamma', 'Gamma', 'Rates', {'VolSpotDynamic': 'StickyDelta', 'RiskSpace': 'Par', 'RiskBuckets': 'Default'}),
    theta = clarion.metric('theta', 'OptionTheta', 'Rates', {'Period': '1'}),
    price_cents = clarion.metric('price_cents', 'Price', 'Rates', {'Unit': 'Cents'}),
    fwd_price_cents = clarion.metric('fwd_price_cents', 'ForwardPrice', 'Rates', {'Unit': 'Cents'}),
    price_bps = clarion.metric('price_bps', 'Price', 'Rates', {'Unit': 'BasisPoints'}),
    price_pct = clarion.metric('price_pct', 'Price', 'Equity', {'Unit': 'PercentLocal'}),
    price_unit = clarion.metric('price_unit', 'Price', 'Equity', {'Unit': 'Local'}),
    spot = clarion.metric('spot', 'Spot', 'Rates'),
    # vega = clarion.metric('vega', 'Vega', 'Rates', {'Period': 'Daily'}),
    # atm_vol_day = clarion.metric('atm_vol_day', 'Volatility', 'Rates', {'Type': 'ATM', 'Period': 'Daily'}),
    atm_vol_ann = clarion.metric('atm_vol_ann', 'Volatility', 'Rates', {'Type': 'ATM', 'Period': 'Annual'}),
    # strike_vol_day = clarion.metric('strike_vol_day', 'Volatility', 'Rates', {'Type': 'Strike', 'Period': 'Daily'}),
    strike_vol_ann = clarion.metric('strike_vol_ann', 'Volatility', 'Rates', {'Type': 'Strike', 'Period': 'Annual'}),
    fx_spot = clarion.metric('fx_spot', 'Spot', 'Fx'),
    fx_fwd = clarion.metric('fx_fwd', 'Forward', 'Fx'),
    # carry_day = clarion.metric('carry_day', 'Carry', 'Credit', {'Horizon': '1d'}),
)

In [12]:
def highlight_cond(df, props, col):
    d = pd.DataFrame(None, df.index, df.columns)
    d.loc[df[col]] = props
    return d 

In [13]:
def highlight_target(df, props=None):
    d = pd.DataFrame(None, df.index, df.columns)
    for index, row in df.iterrows():
        props = 'background-color:red; color: Gold; font-weight: bold;' if row['breach'] else 'color: DarkOrange; font-weight: bold;'
        d.loc[index, ['target', row['var']]] = props
    return d

In [14]:
def send_watch_email(s1, s2):
    from_email = 'gioreports@brevanhoward.com'
    smtp_server = "smtp.rivagecapital.com"
    to_email = [
        'giovambattista.perciaccante@brevanhoward.com',
        'menashe.banit@brevanhoward.com'
    ]

    message = MIMEMultipart("alternative")
    message["Subject"] = f'TRADE WATCH - {pd.Timestamp.now(): %d%b%y %H:%M}'
    message["From"] = from_email
    message["To"] = ','.join(to_email)
    img_tags = [encode_df_as_image_tag(s) for s in [s1, s2]]
    
    html_body = '<br>' + img_tags[0] + ('<br>' * 5) + img_tags[1]
    html_ = MIMEText(html_body, "html")
    message.attach(html_)
    smtp = smtplib.SMTP(smtp_server)
    # smtp.set_debuglevel(1)
    smtp.sendmail(from_email, to_email, message.as_string())

In [15]:
def style_watch(watch):
    style = watch.style.apply(highlight_cond, props='background-color:red; color: white; font-weight: bold;', col='breach', axis=None) \
                .apply(highlight_target, axis=None) \
                .format(na_rep='') \
                .format( 
                    partial(millify, decimal=3, pct=None, zero_rep='-'), 
                    subset=watch.columns[watch.dtypes == 'float64'].drop('target')
                ) \
                .format( 
                    partial(millify, decimal=3, pct=None, zero_rep='0'), 
                    subset='target'
                ) \
                .set_caption('Trades and triggers')
    return style

def style_details(details):
    style = details.style.format(na_rep='').format(
                partial(millify, decimal=3, pct=None, zero_rep='-'), 
                subset=details.columns[details.dtypes == 'float64']
            )
    indices = details.groupby('trade', sort=False).apply(lambda df: df.index[-1])
    style = style.set_table_styles({i: [{'selector': '', 'props': 'border-bottom: 3px solid red;'}] for i in indices}, axis=1).set_caption('Details')
    style = style.hide(subset=details.columns[details.columns.str.startswith('_')], axis=1).set_sticky(axis=1)
    return style

In [16]:
def fn(
    pricer, 
    out1, 
    out2,
    event,
    send_email=True, 
    email_freq=1, 
    send_email_now=False, 
    do_one=False
):
    last_update = time.ctime()
    last_email_sent = time.time()
    p = pricer
    while True:
        if event.is_set():
            break
        try:
            reload(specs)
            from api.watchdog.specs import get_trades, get_targets
            trades = get_trades(clarion)
            targets = get_targets()
            p.get_data(runner, trades, targets, metrics=[*clarion_metrics.values()])
            clear_output()
            last_update = time.ctime()
            s1 = style_watch(p.watch)
            s2 = style_details(p.res)
            # clear_output()
            # out.clear_output(wait=True)
            out1.outputs = ()
            out1.append_display_data(last_update)
            out1.append_display_data(s1)
            out2.outputs = ()
            out2.append_display_data(last_update)
            out2.append_display_data(s2)
            # display(out)
            if send_email:
                if (((time.time() - last_email_sent) / 3600) >= email_freq) or send_email_now:
                    if send_email_now:
                        send_email_now = False
                    print('sending')
                    send_watch_email(s1, s2)
                    last_email_sent = time.time()
            # break
            if not do_one:
                time.sleep(30)
        except Exception as e:
            # return s1
            # raise e
            # clear_output()
            out1.outputs = ()
            out1.append_display_data(f"BROKEN: {last_update}")
            out1.append_display_data(s1)
            out2.outputs = ()
            out2.append_display_data(f"BROKEN: {last_update}")
            out2.append_display_data(s2)
            time.sleep(5)
        if do_one:
            break

In [17]:
p = Pricer()
out_summary = w.Output(layout={'overflow-x': 'auto'})
out_details = w.Output(layout={'overflow-x': 'auto', 'overflow-y': 'auto', 'height': '700px'})
event = threading.Event()

In [18]:
children = [out_summary, out_details]
tab = w.Tab(children=children, titles=['watchdog', 'details'])

In [19]:
thread = threading.Thread(target=fn, kwargs={
    'pricer': p, 
    'out1': out_summary, 
    'out2': out_details,
    'event': event,
    'send_email': True, # False,
    # 'send_email': False,
    'email_freq': 1,
    'send_email_now': True,
})

In [20]:
ss = fn(
    p, 
    out_summary, 
    out_details,  
    event=threading.Event(),
    send_email=False,
    send_email_now=False, 
    do_one=True
) 

### WATCH

#### 

In [24]:
tab

In [25]:
thread.start()

sending


In [ ]:
event.set()

### Send EMAIL

In [ ]:
p.res.to_clipboard()

In [ ]:
p.get_data(runner, trades, targets, metrics=[*clarion_metrics.values()])

In [ ]:
s1 = style_watch(p.watch)
s2 = style_details(p.res)

In [ ]:
send_watch_email(s1, s2)

### TEST

In [ ]:
r.df

In [ ]:
r = Runner(clarion)

In [ ]:
t = [
    clarion.instruments.RatesShortTermFuture(
        currency='USD',
        instrument_name='SOFR 3M CME',
        tenor='Z3',
        notional=1,
    ),
    clarion.instruments.RatesShortTermFuture(
        currency='USD',
        instrument_name='SOFR 3M CME',
        tenor='Z4',
        notional=-1.9,
    ),
    clarion.instruments.RatesShortTermFuture(
        currency='USD',
        instrument_name='SOFR 3M CME',
        tenor='Z5',
        notional=1,
    ),
    clarion.instruments.RatesShortTermFuture(
        currency='EUR',
        instrument_name='EURIBOR 3M ICE',
        tenor='M5',
        notional=1,
    ),
]

In [ ]:
r.run([*clarion_metrics.values()], t)

In [ ]:
r.df

In [117]:
df = p.res.iloc[:, :10]

In [129]:
base_column_defs = {
     'trade_group': {'headerName': "trade", 'showRowGroup': 'trade', 'cellRenderer': 'agGroupCellRenderer', 'resizable': True},
    **{col: {'headerName': col, 'field': col, 'resizable': True} for col in df},
}

In [130]:
base_column_defs['trade'] = {
    **base_column_defs['trade'], 
    'rowGroup': True, 
    'hide': True
}
# base_column_defs['trade_group'] = {'headerName': "trade", 'showRowGroup': 'trade', 'cellRenderer': 'agGroupCellRenderer', 'resizable': True}

In [131]:
grid_options = {
    'columnDefs' : [*base_column_defs.values()],
    'enableSorting': True,
    'enableFilter': True,
    'enableColResize': True,
    'enableRangeSelection': True,
    'suppressColumnVirtualisation': True,  # this flag should allow column resizing when hidden
}

In [133]:
Grid(
    grid_data=df,
    grid_options=grid_options,
    height=700
)

Grid(columns_fit='size_to_fit', compress_data=True, export_mode='disabled', height='700px', menu={'buttons': […

In [27]:
def aggrid(df,color_type='binary',color_columns=[], base_width=140, theme='ag-theme-balham',gradient_percentile=0):
    '''
    :param df: input dataframe
    :param color_type: 'gradient' or 'binary' (i.e. green - red)
    :param base_width: base column width
    :param theme: 'ag-theme-fresh', 'ag-theme-balham', 'ag-theme-material', 'ag-theme-excel', 'ag-theme-alpine'
    :param color_keyword: substring in columns that should be coloured
    :return:
    '''
    # 0. Get numeric and non-numeric columns
    numeric_columns = list(df.select_dtypes(include=np.number).columns)
    non_numeric_columns = list(df.columns.difference(numeric_columns))
    color_columns = color_columns if color_columns else \
    [col for col in numeric_columns if any(substring in col for substring in ['basis','change'])]

    # 1. Define cell coloring schemes (javascript code)

    # # 1.1. Red/Green binary
    # style_binary = """
    # function(params) { if (params.value > 0) { return {color: 'green'}; } else if (params.value < 0) { return {color: 'red'}; } }
    # """

    style_gradient = {}
    style_binary = {}
    #style_black = {}
    for column in numeric_columns:
        # 1.1. White to Green and White to Red gradients for gradient
        s = df[column]
        min_pos = remove_outliers(s[s > 0], percentile=gradient_percentile, fill=0).min()
        max_pos = remove_outliers(s[s > 0], percentile=gradient_percentile).max()

        min_neg = remove_outliers(s[s < 0], percentile=gradient_percentile).min()
        max_neg = remove_outliers(s[s < 0], percentile=gradient_percentile, fill=0).max()

        if 'yield' in column or 'spread' in column:
            sign_pos = '<'
            sign_neg = '>'
            min_pos, max_pos, max_neg, min_neg = max_neg, min_neg, min_pos, max_pos
        else:
            sign_pos = '>'
            sign_neg = '<'

        column_gradient = """
        function(params) { 

        let data_min_pos = """ + f"{min_pos}" + """;
        let data_max_pos = """ + f"{max_pos}" + """;
        let weight_pos = (params.value - data_min_pos)/(data_max_pos - data_min_pos);
        weight_pos = Math.min(Math.max(weight_pos, 0), 1);
        let red_pos = -213 * weight_pos + 255;
        let green_pos = -18 * weight_pos + 255;
        let blue_pos = -209 * weight_pos + 254;

        let data_min_neg = """ + f"{min_neg}" + """;
        let data_max_neg = """ + f"{max_neg}" + """;
        let weight_neg = (params.value - data_min_neg)/(data_max_neg - data_min_neg);
        weight_neg = Math.min(Math.max(weight_neg, 0), 1);
        let red_neg = 30 * weight_neg + 225;
        let green_neg = 165 * weight_neg + 85;
        let blue_neg = 138 * weight_neg + 102;

        let background_color_pos = `rgb(${red_pos}, ${green_pos}, ${blue_pos})`;
        let background_color_neg = `rgb(${red_neg}, ${green_neg}, ${blue_neg})`;

        if (params.value """ + f"{sign_pos}" + """ 0) { return {color: 'black', 'background-color': background_color_pos}; } 
        else if (params.value """ + f"{sign_neg}" + """ 0) { return {color: 'black', 'background-color': background_color_neg}; }
        }
        """
        style_gradient[column] = column_gradient

        # 1.2. Red/Green binary
        column_binary = """
        function(params) { if (params.value """ + f"{sign_pos}" + """ 0) { return {color: 'green'}; } 
        else if (params.value """ + f"{sign_neg}" + """ 0) { return {color: 'red'}; } }
        """
        style_binary[column] = column_binary

    # 1.3. Black
    style_black = """
            function(params) { if (params.value > 0 || params.value < 0)  { return {color: 'black'}; } }
            """

    numeric_style = {column: style_binary[column] if (color_type == 'binary' and column in color_columns)
    else style_gradient[column] if (color_type == 'gradient' and column in color_columns)
    else style_black for column in numeric_columns}

    # 2. Define number formatting / decimal precision (javascript code)

    # 2.1. Percentages
    percent_formatter = """
    function(params) { if (params.value > 0 || params.value < 0) 
                        {return new Intl.NumberFormat('default', {
                                              style: 'percent',
                                              minimumFractionDigits: 2,
                                              maximumFractionDigits: 2,
                                              }).format(params.value);}
                        else {return ' '}
                    }
    """

    # 2.2. Floats
    float_formatter = """
    function(params) {if (params.value > 0 || params.value < 0) 
                        {return new Intl.NumberFormat('default', { maximumFractionDigits: 2 }).format(params.value);} 
                      else {return ' '}}
    """

    numeric_formater = {column : percent_formatter if
    (('yield' in column and 'price' not in column) or 'pct' in column) else float_formatter for column in numeric_columns}

    # 3. Define the column specifications

    numeric_def = [
        {'headerName': column, 'field': column, 'width': base_width, 'rowGroup': 'false',
         'cellStyle': numeric_style[column], 'valueFormatter': numeric_formater[column], 'hide': 'false'}
        for column in numeric_columns
    ]

    non_numeric_def = [
        {'headerName': column, 'field': column, 'width': base_width} for column in non_numeric_columns
    ]

    columnDefs = [*non_numeric_def,*numeric_def] # first non-numeric columns

    for c in columnDefs:
        c["sortable"] = True
        c["filter"] = True
        c["resizable"] = True

    # for the column corresponding to the pivot
    autoGroupColumnDefs = {
        # 'headerName': index_def[0]['headerName'],
        # 'width': 300
    }

    statusBar = {
        'statusPanels': [
            {'statusPanel': 'agTotalAndFilteredRowCountComponent', 'align': 'left'},
            {'statusPanel': 'agTotalRowCountComponent', 'align': 'center'},
            {'statusPanel': 'agFilteredRowCountComponent'},
            {'statusPanel': 'agSelectedRowCountComponent'},
            {'statusPanel': 'agAggregationComponent'},
        ],
    }

    # js_post_grid = ["""
    #     gridOptions.api.context.user_params = view.model.get('user_params');
    # """]

    gridOptions = {
        'pivotMode': 'false',
        'columnDefs': columnDefs,
        #'autoGroupColumnDef': autoGroupColumnDefs,
        #'applyColumnDefOrder': True,
        #'maintainColumnOrder' : 'true',
        'animateRows': 'false',
        'enableRangeSelection': True,
        'rowSelection':'multiple',
        'suppressColumnVirtualisation': True,  # this flag should allow column resizing when hidden
        'statusBar': statusBar,
        #'js_post_grid': js_post_grid
    };

    # 4. Make the Grid

    g = Grid(grid_data=df,
             width="100%",
             height=800,
             grid_options=gridOptions,
             # js_helpers_custom = js_helpers_custom,
             quick_filter=True,
             compress_data=False,  # True
             show_toggle_edit=True,
             sync_on_edit=True,
             export_csv=True,
             export_excel=True,
             theme=theme,  # 'ag-theme-fresh', 'ag-theme-balham'
             show_toggle_delete=True,
             columns_fit='auto',  # 'auto' 'size_to_fit'
             index=False)

    return g

def remove_outliers(x, percentile=5, fill=None):
    if percentile <= 0:
        s = x
    else:
        q = (100 - percentile) / 100
        s = x.loc[(abs(x) < abs(x.quantile(q)))]

    if s.empty:
        s = pd.Series(0)
    if fill == 'ffill':
        return s.ffill()
    elif fill == 'bfill':
        return s.bfill()
    elif isinstance(x, (int, float)) and not isinstance(x, bool):
        return s.fillna(fill)
    return s


In [36]:
df = pd.DataFrame(np.random.normal(loc=1e9, scale=1e6, size=(1000, 20)))

In [37]:
df.columns = df.columns.astype(str)

In [43]:
aggrid(df,color_columns=['2','3'],color_type='gradient',theme='ag-theme-fresh')

Grid(columns_fit='auto', export_mode='disabled', height='800px', menu={'buttons': [{'name': 'Export to CSV', '…

In [44]:
def pivot_grid(df,pivot_field,index,columns,name_dict={},
               color_type='gradient',color_columns=[],format_type=None,base_width=140,theme='ag-theme-fresh'):
    '''
    :param pivot_field: (str) field used when pivoting, e.g. 'pct_change_1d'
    :param index: (ordered) list of variables for the levels of the pivot's index
    :param columns: (ordered) list of variables for the levels of the pivot's columns
    :param name_dict: dictionary to rename the variables
    :param df: input dataframe
    :param color_type: 'gradient' or 'binary' (i.e. green - red)
    :param format_type: 'float' or 'percentage'
    :param base_width: base column width
    :param theme: 'ag-theme-fresh', 'ag-theme-balham', 'ag-theme-material', 'ag-theme-excel', 'ag-theme-alpine'
    :return:
    '''

    # 1. Define cell coloring schemes (javascript code)

    # 1.2. White to Green and White to Red gradients for gradient

    s = df[pivot_field]
    min_pos = remove_outliers(s[s > 0], percentile=5).min()
    max_pos = remove_outliers(s[s > 0], percentile=5).max()

    min_neg = remove_outliers(s[s < 0], percentile=5).min()
    max_neg = remove_outliers(s[s < 0], percentile=5).max()

    if 'yield' in pivot_field or 'spread' in pivot_field:
        sign_pos = '<'
        sign_neg = '>'
        min_pos, max_pos, max_neg, min_neg = max_neg, min_neg, min_pos, max_pos
    else:
        sign_pos = '>'
        sign_neg = '<'

    style_gradient = """
    function(params) { 

    let data_min_pos = """ + f"{min_pos}" + """;
    let data_max_pos = """ + f"{max_pos}" + """;
    let weight_pos = (params.value - data_min_pos)/(data_max_pos - data_min_pos);
    weight_pos = Math.min(Math.max(weight_pos, 0), 1);
    let red_pos = -213 * weight_pos + 255;
    let green_pos = -18 * weight_pos + 255;
    let blue_pos = -209 * weight_pos + 254;

    let data_min_neg = """ + f"{min_neg}" + """;
    let data_max_neg = """ + f"{max_neg}" + """;
    let weight_neg = (params.value - data_min_neg)/(data_max_neg - data_min_neg);
    weight_neg = Math.min(Math.max(weight_neg, 0), 1);
    let red_neg = 30 * weight_neg + 225;
    let green_neg = 165 * weight_neg + 85;
    let blue_neg = 138 * weight_neg + 102;

    let background_color_pos = `rgb(${red_pos}, ${green_pos}, ${blue_pos})`;
    let background_color_neg = `rgb(${red_neg}, ${green_neg}, ${blue_neg})`;

    if (params.value """ + f"{sign_pos}" + """ 0) { return {color: 'black', 'background-color': background_color_pos}; } 
    else if (params.value """ + f"{sign_neg}" + """ 0) { return {color: 'black', 'background-color': background_color_neg}; }
    }
    """
    # 1.1. Red/Green binary
    style_binary = """
    function(params) { if (params.value """ + f"{sign_pos}" + """ 0) { return {color: 'green'}; } 
    else if (params.value """ + f"{sign_neg}" + """ 0) { return {color: 'red'}; } }
    """

    # 1.3. Black
    style_black = """
    function(params) { if (params.value > 0 || params.value < 0)  { return {color: 'black'}; } }
    """

    # 2. Define number formatting / decimal precision (javascript code)

    # 2.1. Percentages
    percent_formatter = """
    function(params) { if (params.value > 0 || params.value < 0) 
                        {return new Intl.NumberFormat('default', {
                                              style: 'percent',
                                              minimumFractionDigits: 2,
                                              maximumFractionDigits: 2,
                                              }).format(params.value);}
                        else {return ' '}
                    }
    """

    # 2.2. Floats
    float_formatter = """
    function(params) {if (params.value > 0 || params.value < 0) 
                        {return new Intl.NumberFormat('default', { maximumFractionDigits: 3 }).format(params.value);} 
                      else {return ' '}}
    """

    # 3. Define the column / specifications
    if format_type is None:
        #format_type = 'percentage' if any(substring in pivot_field for substring in ['pct']) else 'float'
        format_type = 'percentage' if \
            (any(substring in pivot_field for substring in ['pct','yield']) and 'yield_change' not in pivot_field) \
            else 'float'


    cell_style = style_binary if (color_type == 'binary' and any(substring in pivot_field for substring in ['basis','change'])) \
        else style_gradient if (color_type == 'gradient' and any(substring in pivot_field for substring in ['basis','change'])) else style_black
    value_formatter = float_formatter if format_type == 'float' else percent_formatter if format_type == 'percentage' else float_formatter


    index_def= [
        {'headerName': name_dict[field] if name_dict else field, 'field': field, 'width': base_width, 'rowGroup': 'true'} for field in index
    ]

    column_def = [
        {'headerName': name_dict[field] if name_dict else field, 'field': field, 'width': base_width,
         'pivot': 'true', 'enablePivot': True} for field in columns
    ]

    value_def = [
        {'headerName': pivot_field, 'field': pivot_field, 'width': base_width, 'aggFunc': 'avg',
         'cellStyle': cell_style, 'valueFormatter': value_formatter, 'hide': 'true'}
    ]

    columnDefs = [*index_def, *column_def, *value_def]

    for c in columnDefs:
        c["sortable"] = True
        c["filter"] = True
        c["resizable"] = True

    # for the column corresponding to the pivot
    autoGroupColumnDefs = {
        'headerName': value_def[0]['headerName'],
        'width': 300,
        # 'cellRendererParams':{
        #     'suppressCount': 'true',
        #     'checkbox':'true'
        # }
    }

    gridOptions = {
        'pivotMode': 'true',
        'columnDefs': columnDefs,
        'autoGroupColumnDef': autoGroupColumnDefs,
        'animateRows': 'true',
        'enableRangeSelection': True,
        'suppressColumnVirtualisation': True  # this flag should allow column resizing when hidden
    };

    # 4. Make the Grid

    g = Grid(grid_data=df,
             width="100%",
             height=800,
             grid_options=gridOptions,
             # js_helpers_custom = js_helpers_custom,
             quick_filter=True,
             compress_data=False,  # True
             show_toggle_edit=True,
             sync_on_edit=True,
             export_csv=True,
             export_excel=True,
             theme=theme,  # 'ag-theme-fresh', 'ag-theme-balham'
             show_toggle_delete=True,
             columns_fit='auto',  # 'auto' 'size_to_fit'
             index=False)

    return g
